In [16]:
!pip install tensorflow-gpu

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


# Get Directory

In [17]:
import os
import pandas as pd
import json

cwd = os.getcwd()

#Main Program

In [18]:
import io
import pandas as pd
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

In [19]:
speaker_df = pd.read_csv(cwd + '/Data/speaker-data.csv')
speaker_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 32 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Unnamed: 0                           10000 non-null  int64  
 1   profile_pic_url                      10000 non-null  object 
 2   full_name                            10000 non-null  object 
 3   occupation                           8939 non-null   object 
 4   headline                             9396 non-null   object 
 5   summary                              6276 non-null   object 
 6   country_full_name                    10000 non-null  object 
 7   city                                 9663 non-null   object 
 8   1st Recent Experience in Years       8939 non-null   float64
 9   2nd Recent Experience in Years       8117 non-null   float64
 10  3rd Recent Experience in Years       7464 non-null   float64
 11  1st Recent Experience Title  

# Using Regex to get Categories

In [20]:
import pandas as pd
import re

# Load the CSV file
data = data = pd.DataFrame(speaker_df.iloc[:,[1,2,3,4,5,8,11]])

# Clean up column names
data = data.rename(columns={'1st Recent Experience Title': 'Experience'})

# Convert all entries in the 'Text' column to string and handle NaNs
data['Experience'] = data['Experience'].astype(str).fillna('')

# Define the categories and their keywords
keywords = {
    'Business': ['business', 'consultant', 'marketing', 'sales', 'finance', 'management', 'executive'],
    'Entertainment': ['actor', 'artist', 'entertainment', 'musician', 'theater', 'film', 'play', 'concert'],
    'Politics': ['politician', 'government', 'policy', 'political', 'election', 'campaign'],
    'Sport': ['sport', 'athlete', 'coach', 'fitness', 'football', 'basketball', 'soccer', 'baseball', 'olympic'],
    'Tech': ['tech', 'technology', 'software', 'hardware', 'engineer', 'IT', 'developer', 'data', 'cyber'],
    'Healthcare': ['health', 'medical', 'doctor', 'nurse', 'hospital', 'clinic', 'healthcare', 'therapist', 'pharmaceutical'],
    'Academic': ['professor', 'teacher', 'academic', 'education', 'research', 'scientist', 'lecturer', 'scholar', 'study', 'student', 'university', 'college', 'undergraduate'],
    'Media & News': ['media', 'journalist', 'reporter', 'editor', 'news', 'broadcast', 'press', 'breaking', 'headline', 'journalism', 'bulletin'],

}

# Function to get top 3 categories
def top_3_categories(text):
    text = text.lower()
    scores = {category: 0 for category in keywords}
    for category, words in keywords.items():
        for word in words:
            scores[category] += len(re.findall(rf'\b{word}\b', text))
    # Sort by score and get the top 3 categories
    top_categories = sorted(scores, key=scores.get, reverse=True)[:3]
    # Replace empty top categories with 'Others' if none match the criteria
    top_categories = [cat if scores[cat] > 0 else float('NaN') for cat in top_categories]
    return top_categories

# Apply the function to the dataset and expand into separate columns
data[['Category 1', 'Category 2', 'Category 3']] = data['Experience'].apply(top_3_categories).apply(pd.Series)

data = data[~data['Category 1'].isna()].reset_index().iloc[:,1:]

# Save the labeled data
output_path = cwd + '/Data/labeled_data.csv'
data.to_csv(output_path, index=False)

# Display the labeled data
data.head()


,profile_pic_url,full_name,occupation,headline,summary,1st Recent Experience in Years,Experience,Category 1,Category 2,Category 3
0,https://media-exp1.licdn.com/dms/image/C4E03AQ...,Andrew Bailey,Science & Technology Counsel at National Pork ...,National Pork Producers Council,Experienced government affairs and public poli...,4.553425,Science & Technology Counsel,Tech,NaN,NaN
1,https://media-exp1.licdn.com/dms/image/C4E03AQ...,Alex Cabrera,Research Intern at Apple,PhD Student at Carnegie Mellon University,NaN,0.416438,Research Intern,Academic,NaN,NaN
2,https://media-exp1.licdn.com/dms/image/C4E03AQ...,Alcino de Oliveira Manuel,Marketing Consultant at Alcino Media LLC,Founder @ AlcinoMedia | Facebook/Meta & Google...,"I use the drive and commitment to help, inspir...",3.216438,Marketing Consultant,Business,NaN,NaN
3,https://media-exp1.licdn.com/dms/image/C4D03AQ...,Ashleigh Ann Gardner,Actor and Playwright at Self-employed,Actor at Universal Orlando Resort,"Writer, research professional, and performer s...",9.556164,Actor and Playwright,Entertainment,NaN,NaN
4,https://media-exp1.licdn.com/dms/image/C4E03AQ...,Abdullah Al Mamun,TD Process integration engineer at Intel Corpo...,TD Process integration engineer at Intel Corpo...,🔲Process integration engineer in BE interconne...,4.471233,TD Process integration engineer,Tech,NaN,NaN


In [21]:
data

,profile_pic_url,full_name,occupation,headline,summary,1st Recent Experience in Years,Experience,Category 1,Category 2,Category 3
0,https://media-exp1.licdn.com/dms/image/C4E03AQ...,Andrew Bailey,Science & Technology Counsel at National Pork ...,National Pork Producers Council,Experienced government affairs and public poli...,4.553425,Science & Technology Counsel,Tech,NaN,NaN
1,https://media-exp1.licdn.com/dms/image/C4E03AQ...,Alex Cabrera,Research Intern at Apple,PhD Student at Carnegie Mellon University,NaN,0.416438,Research Intern,Academic,NaN,NaN
2,https://media-exp1.licdn.com/dms/image/C4E03AQ...,Alcino de Oliveira Manuel,Marketing Consultant at Alcino Media LLC,Founder @ AlcinoMedia | Facebook/Meta & Google...,"I use the drive and commitment to help, inspir...",3.216438,Marketing Consultant,Business,NaN,NaN
3,https://media-exp1.licdn.com/dms/image/C4D03AQ...,Ashleigh Ann Gardner,Actor and Playwright at Self-employed,Actor at Universal Orlando Resort,"Writer, research professional, and performer s...",9.556164,Actor and Playwright,Entertainment,NaN,NaN
4,https://media-exp1.licdn.com/dms/image/C4E03AQ...,Abdullah Al Mamun,TD Process integration engineer at Intel Corpo...,TD Process integration engineer at Intel Corpo...,🔲Process integration engineer in BE interconne...,4.471233,TD Process integration engineer,Tech,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3740,https://media-exp1.licdn.com/dms/image/C5603AQ...,Aaron Dodgson,Sr. Account Executive at Sprinklr,Sr. Account Executive at Sprinklr,NaN,3.800000,Sr. Account Executive,Business,NaN,NaN
3741,https://static-exp1.licdn.com/sc/h/244xhbkr7g4...,Aaron Saul,Software Engineer at Daugherty Business Solutions,Software Engineer,Software Engineer with 9 years of experience d...,2.049315,Software Engineer,Tech,NaN,NaN
3742,https://media-exp1.licdn.com/dms/image/C5603AQ...,Aaron Dolgin,Communications System Engineer at Northrop Gru...,Communications System Engineer at Northrop Gru...,Experienced Electrical engineer Specialist wit...,2.550685,Communications System Engineer,Tech,NaN,NaN
3743,https://media-exp1.licdn.com/dms/image/C4E03AQ...,Aaron Doliana,Language Engineer at Amazon Web Services (AWS),Language Engineer at Amazon Web Services,Language scientist with 10+ years of experienc...,2.131507,Language Engineer,Tech,NaN,NaN
